In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, FloatSlider, Button, VBox, Output
from IPython.display import display, clear_output
import time

University facilities data

In [ ]:
facilities = ["Maintenance Hub", "Lecture Hall", "Library", "Dorm", "Labs"]
num_facilities = len(facilities)
coordinates = np.array([[0, 0], [2, 3], [-1, 2], [3, -1], [-2, -2]])  # Approximate coordinates
dist_graph = np.array([
    [0, 300, 200, 400, 250],  # Maintenance Hub
    [300, 0, 250, 350, 400],  # Lecture Hall
    [200, 250, 0, 300, 200],  # Library
    [400, 350, 300, 0, 350],  # Dorm
    [250, 400, 200, 350, 0]   # Labs
])
priorities = np.array([0.0, 0.8, 0.3, 0.5, 0.2])  # Priority for each facility (e.g., Lecture Hall has high priority)

Initialize random seed

In [ ]:
seed = 42
np.random.seed(seed)

Compute fitness considering priorities

In [ ]:
def compute_fitness(solution, dist_graph, priorities, priority_weight):
    total_dist = 0
    priority_factor = 0
    for i in range(len(solution) - 1):
        total_dist += dist_graph[solution[i]][solution[i + 1]]
        if i < len(solution) - 1:  # Exclude the return to start
            priority_factor += priorities[solution[i + 1]]
    # Incorporate priority: shorter distance and higher priority are better
    adjusted_dist = total_dist / (1 + priority_weight * priority_factor)
    return 1 / adjusted_dist  # Higher fitness for shorter adjusted distance

Generate a random route

In [ ]:
def generate_route(num_facilities, start_facility=0):
    route = [start_facility] + list(np.random.permutation(list(range(1, num_facilities))))
    route.append(start_facility)
    return np.array(route)

Perturb a route by swapping two facilities (excluding start/end)

In [ ]:
def perturb_route(route):
    route_inner = route[1:-1].copy()  # Exclude start and end
    idx1, idx2 = np.random.choice(len(route_inner), 2, replace=False)
    route_inner[idx1], route_inner[idx2] = route_inner[idx2], route_inner[idx1]
    return np.concatenate(([route[0]], route_inner, [route[-1]]))

ABC Algorithm

In [ ]:
def run_abc(colony_size, max_iters, limit, priority_weight):
    # Initialize population
    population = [generate_route(num_facilities) for _ in range(colony_size // 2)]
    fitness = [compute_fitness(sol, dist_graph, priorities, priority_weight) for sol in population]
    trials = [0] * len(population)

    best_solution = population[np.argmax(fitness)]
    best_fitness = max(fitness)
    best_length = sum(dist_graph[best_solution[i]][best_solution[i + 1]] for i in range(len(best_solution) - 1))

    # Main loop
    for _ in range(max_iters):
        # Employed Bees Phase
        for i in range(len(population)):
            new_solution = perturb_route(population[i])
            new_fitness = compute_fitness(new_solution, dist_graph, priorities, priority_weight)
            if new_fitness > fitness[i]:
                population[i] = new_solution
                fitness[i] = new_fitness
                trials[i] = 0
            else:
                trials[i] += 1

        # Onlooker Bees Phase
        fitness_sum = sum(fitness)
        if fitness_sum == 0:
            probabilities = [1 / len(fitness)] * len(fitness)
        else:
            probabilities = [f / fitness_sum for f in fitness]

        for _ in range(len(population)):
            i = np.random.choice(len(population), p=probabilities)
            new_solution = perturb_route(population[i])
            new_fitness = compute_fitness(new_solution, dist_graph, priorities, priority_weight)
            if new_fitness > fitness[i]:
                population[i] = new_solution
                fitness[i] = new_fitness
                trials[i] = 0
            else:
                trials[i] += 1

        # Scout Bees Phase
        for i in range(len(population)):
            if trials[i] >= limit:
                population[i] = generate_route(num_facilities)
                fitness[i] = compute_fitness(population[i], dist_graph, priorities, priority_weight)
                trials[i] = 0

        # Update best solution
        current_best_idx = np.argmax(fitness)
        current_best_fitness = fitness[current_best_idx]
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_solution = population[current_best_idx]
            best_length = sum(dist_graph[best_solution[i]][best_solution[i + 1]] for i in range(len(best_solution) - 1))

    return best_solution, best_length


Plot function

In [ ]:
def plot_solution(best_solution, best_length):
    plt.clf()
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(coordinates[:, 0], coordinates[:, 1], c='red', s=100)
    for i, (x, y) in enumerate(coordinates):
        ax.text(x + 0.1, y, facilities[i], fontsize=12)

    for i in range(num_facilities):
        start = coordinates[best_solution[i]]
        end = coordinates[best_solution[i + 1]]
        ax.plot([start[0], end[0]], [start[1], end[1]], 'b-')

    ax.set_title(f"Best Route: {best_length:.2f} meters\nPath: {' -> '.join([facilities[i] for i in best_solution])}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)
    plt.show()


GUI Widgets

In [ ]:
colony_size_slider = IntSlider(min=2, max=50, value=10, description='Colony Size')
max_iters_slider = IntSlider(min=1, max=100, value=20, description='Max Iterations')
limit_slider = IntSlider(min=1, max=20, value=5, description='Limit')
priority_weight_slider = FloatSlider(min=0.0, max=2.0, value=1.0, description='Priority Weight')
run_button = Button(description="Run ABC")

output = Output()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print("Running ABC...")
        start_time = time.time()
        best_solution, best_length = run_abc(
            colony_size=colony_size_slider.value,
            max_iters=max_iters_slider.value,
            limit=limit_slider.value,
            priority_weight=priority_weight_slider.value
        )
        print(f"ABC finished in {time.time() - start_time:.2f} seconds.")
        plot_solution(best_solution, best_length)

run_button.on_click(on_button_clicked)

Display GUI

In [ ]:
display(VBox([
    colony_size_slider, max_iters_slider, limit_slider,
    priority_weight_slider, run_button, output
]))